## 範例目標

* 了解如何計算傳統電腦視覺特徵 hog
* 了解如何利用 opencv 調用 SVM

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.astype(int)
y_test = y_test.astype(int)

#### 產生 Hist 特徵的訓練資料

In [0]:
import cv2
import numpy as np

x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    hist_feature = []
    for j in range(3):
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([x_train[i]], [j], None, [16], [0, 256])
        hist_feature.extend(hist.flatten())
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    hist_feature = []
    for j in range(3):
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([x_test[i]], [j], None, [16], [0, 256])
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)

#### SVM model
* SVM 是機器學習中一個經典的分類算法，這裡直接調用 opencv 中實現好的函數

[支持向量机(SVM)是什么意思？](https://www.zhihu.com/question/21094489)

In [56]:
# 用 histogram 特徵訓練 SVM 模型
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(0.5)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(30)

# training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train_pred = SVM_hist.predict(x_train_histogram)
_, y_hist_test_pred = SVM_hist.predict(x_test_histogram)

# evaluation
from sklearn.metrics import accuracy_score

hist_train_acc = accuracy_score(y_train, y_hist_train_pred)
hist_test_acc = accuracy_score(y_test, y_hist_test_pred)
print(f"hist_train_acc: {hist_train_acc:.5f}")
print(f"hist_test_acc: {hist_test_acc:.5f}")

hist_train_acc: 0.12078
hist_test_acc: 0.12280


#### 產生 HOG 特徵的訓練資料
* HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵

[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)

In [0]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

In [54]:
# 用 HOG 特徵訓練 SVM 模型
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(0.5)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(30)

# training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train_pred = SVM_hog.predict(x_train_hog)
_, y_hog_test_pred = SVM_hog.predict(x_test_hog)

# evaluation
from sklearn.metrics import accuracy_score

hog_train_acc = accuracy_score(y_train, y_hog_train_pred)
hog_test_acc = accuracy_score(y_test, y_hog_test_pred)
print(f"hog_train_acc: {hog_train_acc:.5f}")
print(f"hog_test_acc: {hog_test_acc:.5f}")

hog_train_acc: 0.21388
hog_test_acc: 0.21800
